# CIS 390 - Supervised Machine Learning Final Project
For the first step of this project, I'll be demonstrating the "Data Preparation" step of the CRISP-DM process about the **"Predicting Motorcycle Resale Prices" Use Case**. In this jupyter notebook file, I'll go through displaying general information about the dataset, preparing a data quality report, cleaning the data, exploratory data analysis, handle missing data, feature engineering, and detecting outliers. When finished, we will save the resulting `DataFrame` to a file to use in our other `.ipynb` file for this project (the one where we actually build and evaluate the model).

## Learning About The Dataset
To start off, I'll load required libraries for this project, load in the dataset, and learn about it. This will include creating a data quality report and seeing what exactly we're working with.

In [1]:
import pandas as pd
import numpy as np

In [3]:
# I took the original "Motorcycles.txt" data and converted it to csv format by making the first line the column names
# and removing the lines starting with "%" and "@"
df = pd.read_csv("Data/Motorcycles.csv")
df.head()

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,'Royal Enfield Classic 350',175000,2019,Individual,'1st owner',350,?
1,'Honda Dio',45000,2017,Individual,'1st owner',5650,?
2,'Royal Enfield Classic Gunmetal Grey',150000,2018,Individual,'1st owner',12000,148114.0
3,'Yamaha Fazer FI V 2.0 [2016-2018]',65000,2015,Individual,'1st owner',23000,89643.0
4,'Yamaha SZ [2013-2014]',20000,2011,Individual,'2nd owner',21000,?


In [3]:
# Data Quality Report
df.replace("?", np.nan, inplace=True)
missing_values = df.isnull().sum()
summary_stats = df.describe(include="all")
data_types = df.dtypes
categorical_columns = df.select_dtypes(include=["object"]).columns
unique_values = {col: df[col].nunique() for col in categorical_columns}

print("Missing Values:\n", missing_values)
print("\nSummary Statistics:\n", summary_stats)
print("\nData Types:\n", data_types)
print("\nCategorical Columns:", categorical_columns)
print("\nUnique Values in Categorical Columns:\n", unique_values)

Missing Values:
 name                   0
selling_price          0
year                   0
seller_type            0
owner                  0
km_driven              0
ex_showroom_price    435
dtype: int64

Summary Statistics:
                       name  selling_price         year seller_type  \
count                 1061    1061.000000  1061.000000        1061   
unique                 279            NaN          NaN           2   
top     'Bajaj Pulsar 150'            NaN          NaN  Individual   
freq                    41            NaN          NaN        1055   
mean                   NaN   59638.151744  2013.867107         NaN   
std                    NaN   56304.291973     4.301191         NaN   
min                    NaN    5000.000000  1988.000000         NaN   
25%                    NaN   28000.000000  2011.000000         NaN   
50%                    NaN   45000.000000  2015.000000         NaN   
75%                    NaN   70000.000000  2017.000000         NaN   
max

## Preparing The Data
To prepare the dataset for building a machine learning model to predict resale price, we need to thoroughly clean and preprocess the data. Let's first understand that the target variable is `selling_price`. This column must:
   - contain numeric data
   - have no missing or invalid values
   - be free from extreme outliers
   
We also have to handle missing data, which only appears in the `ex_showroom_price` column. We can impute these missing values using the median. We will also perform feature engineering, detect and handle extreme outliers, as well as handle categorical variables by encoding them in this step.

In [4]:
# Handle missing data
df['ex_showroom_price'].fillna(df['ex_showroom_price'].median(), inplace=True)

# Feature engineering
current_year = 2024
df['age'] = current_year - df['year']

df['usage_intensity'] = df['km_driven'] / df['age']
df['usage_intensity'] = df['usage_intensity'].replace([np.inf, -np.inf], 0)  # handle potential division by 0

df['brand'] = df['name'].apply(lambda x: str(x).split()[0][1:])  # get first word of name column

# Convert categorical variables to numerical
df['ex_showroom_price'] = pd.to_numeric(df['ex_showroom_price'], errors='coerce')
df['seller_type'] = df['seller_type'].astype('category').cat.codes
df['owner'] = df['owner'].astype('category').cat.codes
df['brand'] = df['brand'].astype('category').cat.codes

# Detect outliers
numeric_columns = ['selling_price', 'km_driven', 'ex_showroom_price', 'age', 'usage_intensity']
# Let's get a before and after of removing outliers to see what's changing...
print("Before:", df.describe(include="all"))
for col in numeric_columns:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df = df[(df[col] >= lower_bound) * (df[col] <= upper_bound)]
print("\nAfter:", df.describe(include="all"))

Before:                       name  selling_price         year  seller_type  \
count                 1061    1061.000000  1061.000000  1061.000000   
unique                 279            NaN          NaN          NaN   
top     'Bajaj Pulsar 150'            NaN          NaN          NaN   
freq                    41            NaN          NaN          NaN   
mean                   NaN   59638.151744  2013.867107     0.994345   
std                    NaN   56304.291973     4.301191     0.075022   
min                    NaN    5000.000000  1988.000000     0.000000   
25%                    NaN   28000.000000  2011.000000     1.000000   
50%                    NaN   45000.000000  2015.000000     1.000000   
75%                    NaN   70000.000000  2017.000000     1.000000   
max                    NaN  760000.000000  2020.000000     1.000000   

              owner      km_driven  ex_showroom_price          age  \
count   1061.000000    1061.000000       1.061000e+03  1061.000000   

## Interpreting And Saving The Results
There are some interesting discoveries to be taken away from this before and after. One of which is how much the mean selling price decreased after removing outliers. This shows that this number was being hiked up from some extremely high values in the dataset. We will delve into these observations more in the synopsis document, but now that we have successfully prepared the data for modeling, we will save it to another file which will then be used in the `FinalProject_Modeling.ipynb` file. 

In [5]:
df.to_csv("abt_motorcycle_prices.csv", index=False)